In [1]:
import ta
import numpy as np
import warnings
from datetime import datetime
import pandas as pd
import MetaTrader5 as mt5
warnings.filterwarnings("ignore")
mt5.initialize()
from Chapter_08_MT5 import *


def sr_sma_rsi(symbol):
    
    # Import / Features engineering
    df = MT5.get_data(symbol, 100, timeframe=mt5.TIMEFRAME_M1)
    
    
    # Support and resistance building
    df["support"] = np.nan
    df["resistance"] = np.nan

    df.loc[(df["low"].shift(5) > df["low"].shift(4)) &
        (df["low"].shift(4) > df["low"].shift(3)) &
        (df["low"].shift(3) > df["low"].shift(2)) &
        (df["low"].shift(2) > df["low"].shift(1)) &
        (df["low"].shift(1) > df["low"].shift(0)), "support"] = df["low"]


    df.loc[(df["high"].shift(5) < df["high"].shift(4)) &
    (df["high"].shift(4) < df["high"].shift(3)) &
    (df["high"].shift(3) < df["high"].shift(2)) &
    (df["high"].shift(2) < df["high"].shift(1)) &
    (df["high"].shift(1) < df["high"].shift(0)), "resistance"] = df["high"]


    # Create Simple moving average 30 days
    df["SMA fast"] = df["close"].rolling(30).mean()

    # Create Simple moving average 60 days
    df["SMA slow"] = df["close"].rolling(60).mean()

    df["rsi"] = ta.momentum.RSIIndicator(df["close"], window=10).rsi()

    # RSI yersteday
    df["rsi yersteday"] = df["rsi"].shift(1)

    # Create the signal
    df["signal"] = 0

    df["smooth resistance"] = df["resistance"].fillna(method="ffill")
    df["smooth support"] = df["support"].fillna(method="ffill")


    condition_1_buy = (df["close"].shift(1).iloc[-1] < df["smooth resistance"].shift(1).iloc[-1]) & \
                    (df["smooth resistance"].iloc[-1]*(1+0.5/100) < df["close"].iloc[-1])
    condition_2_buy = df["SMA fast"].iloc[-1] > df["SMA slow"].iloc[-1]

    condition_3_buy = df["rsi"].iloc[-1] < df["rsi yersteday"].iloc[-1]

    condition_1_sell = (df["close"].shift(1).iloc[-1] > df["smooth support"].shift(1).iloc[-1]) & \
                    (df["smooth support"].iloc[-1]*(1+0.5/100) > df["close"].iloc[-1])
    condition_2_sell = df["SMA fast"].iloc[-1] < df["SMA slow"].iloc[-1]

    condition_3_sell = df["rsi"].iloc[-1] > df["rsi yersteday"].iloc[-1]



    buy = condition_1_buy and condition_2_buy and condition_3_buy
    sell = condition_1_sell and condition_2_sell and condition_3_sell

    return buy,sell

mt5.initialize()
# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")


info_order = {
    "Bitcoin": ["BTCUSD", 1.],
    "Monero": ["XMRUSD", 0.1],
    "Ethereum": ["ETHUSD", 0.3],
    "Ethereum Classic": ["ETCUSD", 1.],
    "Zcash": ["ZECUSD", 1.],
    
}


start = datetime.now().strftime("%H:%M:%S") #"23:59:59"
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,6): # If you want to trade only from Monday to Friday
        is_time = datetime.now().strftime("%H:%M:%S") == start 
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = sr_sma_rsi(symbol)

             # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")

------------------------------------------------------------------
Date:  2022-03-05 18:48:14
Balance: 10000.0 USD, 	Equity: 10000.0 USD, 	Profit: 0.0 USD
------------------------------------------------------------------


KeyboardInterrupt: 